### Shared Word

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 0  # 28  # native word end index

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False
fourgram_sentence_check = False
fivegram_sentence_check = False

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True

In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

In [5]:
def remove_repetition(word_group):
    '''
    remove_repetition(word_group): detect word repetion in word group 
    '''
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
1005970,karnaya,5
1005971,dörtlümüzün,5
1005972,karnavalınız,5
1005973,hurmanın,5


In [7]:
df_word = df_word_all.iloc[word_start:word_end,]
df_word

,word,frequency


In [8]:
#lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
#lang_pair_list

In [9]:
#df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,english_word
0,abaküs,abacus
1,abandone,abandon
2,abanoz,ebony
3,abdomen,abdomen
4,abluka,blockade
...,...,...
2068,zodyak,zodiac
2069,zombi,zombie
2070,zooloji,zoology
2071,zoolojik,zoological


In [10]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel("Turkish_English_Shared_Word_Prefix_Suffix_Custom.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass
df_pair

,dict_entry_main,english_word
0,abaküs,abacus
1,abaküsü,abacus
2,abaküsüm,abacus
3,abaküsünü,abacus
4,abaküsle,abacus
...,...,...
41284,zumda,zoom
41285,zumlama,zoom
41286,zumwalt,zoom
41287,zumi,zoom


In [11]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [12]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [13]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

In [14]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three", "Four", "Five", "Sentence"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}.xlsx", index=False)    

#### Concat Result

In [15]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,bloklama
1,paralelde
2,ramlada
3,asra
4,boar
...,...
39405,fobin
39406,vampirates
39407,ültimatomuna
39408,ramayla


In [16]:
# Option For Frequency
if shared_word_frequency:
    df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
    df_word = pd.merge(df_word,df_word_all, how="inner", on="word")
    df_word.drop_duplicates(inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    pass

df_word

,word,frequency
0,bloklama,18
1,paralelde,77
2,ramlada,6
3,asra,126
4,boar,51
...,...,...
39635,fobin,85
39636,vampirates,5
39637,ültimatomuna,11
39638,ramayla,5


In [17]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,bana biraz,9044
1,milyon dolar,6377
2,biraz fazla,5111
3,bana şans,2722
4,bas gaza,2394
...,...,...
18099,basa müzikle,6
18100,kabininde izole,6
18101,bas solodan,6
18102,bas tabutunu,6


In [18]:
# Option For Twogram Sentence
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram_sent = df_twogram_sent.loc[:,["twogram"]]
    df_shared_twogram = pd.merge(df_shared_twogram, df_twogram_sent, how="inner", on="twogram")
    df_shared_twogram.drop_duplicates(inplace=True)
    df_shared_twogram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_twogram


,twogram,freq_twogram
0,bana biraz,9044
1,milyon dolar,6377
2,biraz fazla,5111
3,bana şans,2722
4,bas gaza,2394
...,...,...
18099,basa müzikle,6
18100,kabininde izole,6
18101,bas solodan,6
18102,bas tabutunu,6


In [19]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,milyon dolardan fazla,279
1,standart kablolu orkestrası,191
2,bana biraz şeker,176
3,bana telefon numaranı,144
4,bana biraz fazla,118
...,...,...
617,filmin psikolojik mesajı,10
618,filmi çarşıda çekmek,10
619,filmleri televizyonu radyoyu,10
620,personel şefi morganla,10


In [20]:
# Option For Threegram Sentence
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram_sent = df_twogram_sent.loc[:,["threegram"]]
    df_shared_threegram = pd.merge(df_shared_threegram, df_threegram_sent, how="inner", on="threegram")
    df_shared_threegram.drop_duplicates(inplace=True)
    df_shared_threegram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_threegram

,threegram,freq_threegram
0,milyon dolardan fazla,279
1,standart kablolu orkestrası,191
2,bana biraz şeker,176
3,bana telefon numaranı,144
4,bana biraz fazla,118
...,...,...
617,filmin psikolojik mesajı,10
618,filmi çarşıda çekmek,10
619,filmleri televizyonu radyoyu,10
620,personel şefi morganla,10


In [21]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram
0,ajan romanoff doktor bannera,30
1,kredi kartları sivil frekans,23
2,fazla kaliteli normal banka,20
3,servis modülü reaksiyon kontrol,20
4,müslüman türkler avrupada hıristiyanlık,17
...,...,...
63,biraz müzik biraz dans,8
64,orkestra şefi bana pasta,7
65,numarası amberin video ekipmanına,7
66,kupa kahvenin kolon kanserini,7


In [22]:
# Option For Fourgram Sentence
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fourgram_sent = df_twogram_sent.loc[:,["fourgram"]]
    df_shared_fourgram = pd.merge(df_shared_fourgram, df_fourgram_sent, how="inner", on="fourgram")
    df_shared_fourgram.drop_duplicates(inplace=True)
    df_shared_fourgram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_fourgram

,fourgram,freq_fourgram
0,ajan romanoff doktor bannera,30
1,kredi kartları sivil frekans,23
2,fazla kaliteli normal banka,20
3,servis modülü reaksiyon kontrol,20
4,müslüman türkler avrupada hıristiyanlık,17
...,...,...
63,biraz müzik biraz dans,8
64,orkestra şefi bana pasta,7
65,numarası amberin video ekipmanına,7
66,kupa kahvenin kolon kanserini,7


In [23]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram
0,krizli ülserli prostat kanserli lösemili,16
1,alfa alfa bravo eko çarli,13
2,telefon santrali postane elektrik santrali,12
3,cd radyo televizyon telefon bar,10
4,radyo televizyon telefon bar video,10
5,servis modülü reaksiyon kontrol sistemi,10
6,modülü reaksiyon kontrol sistemi valfleri,10


In [24]:
# Option For Fivegram Sentence
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fivegram_sent = df_twogram_sent.loc[:,["fivegram"]]
    df_shared_fivegram = pd.merge(df_shared_fivegram, df_fivegram_sent, how="inner", on="fivegram")
    df_shared_fivegram.drop_duplicates(inplace=True)
    df_shared_fivegram.reset_index(drop=True, inplace=True)
else:
    pass

df_shared_fivegram

,fivegram,freq_fivegram
0,krizli ülserli prostat kanserli lösemili,16
1,alfa alfa bravo eko çarli,13
2,telefon santrali postane elektrik santrali,12
3,cd radyo televizyon telefon bar,10
4,radyo televizyon telefon bar video,10
5,servis modülü reaksiyon kontrol sistemi,10
6,modülü reaksiyon kontrol sistemi valfleri,10


In [25]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,bas gaza,1785
1,gaza bas,759
2,telefon numarası,232
3,biraz komik,229
4,kredi kartı,208
...,...,...
1575,eroin trajedisi,6
1576,elektronik teyp,6
1577,fondip kovboy,6
1578,kamp karavanı,6


In [26]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,bloklama,18,bana biraz,9044.0,milyon dolardan fazla,279.0,ajan romanoff doktor bannera,30.0,krizli ülserli prostat kanserli lösemili,16.0,bas gaza,1785.0
1,paralelde,77,milyon dolar,6377.0,standart kablolu orkestrası,191.0,kredi kartları sivil frekans,23.0,alfa alfa bravo eko çarli,13.0,gaza bas,759.0
2,ramlada,6,biraz fazla,5111.0,bana biraz şeker,176.0,fazla kaliteli normal banka,20.0,telefon santrali postane elektrik santrali,12.0,telefon numarası,232.0
3,asra,126,bana şans,2722.0,bana telefon numaranı,144.0,servis modülü reaksiyon kontrol,20.0,cd radyo televizyon telefon bar,10.0,biraz komik,229.0
4,boar,51,bas gaza,2394.0,bana biraz fazla,118.0,müslüman türkler avrupada hıristiyanlık,17.0,radyo televizyon telefon bar video,10.0,kredi kartı,208.0
...,...,...,...,...,...,...,...,...,...,...,...,...
39635,fobin,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39636,vampirates,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39637,ültimatomuna,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39638,ramayla,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx", index=False)

##### Concat Result With Comma

In [28]:
def word_in_wordgroup(df, list_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            #word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(10)  # Option
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{list_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [29]:
df_word_order_twogram = word_in_wordgroup(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_all, "word", "sentence") 

In [30]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()

In [31]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [32]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence
0,abanoz,stres abanoz,NaN,NaN,NaN,NaN
1,abanozun,doktor abanozun,NaN,NaN,NaN,NaN
2,abluka,abluka riskini,NaN,NaN,NaN,NaN
3,ablukaya,bankanızı ablukaya,NaN,NaN,NaN,NaN
4,ablukayı,"asla ablukayı, destroyeri ablukayı",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
8157,zumlu,NaN,NaN,NaN,NaN,zumlu objektif
8158,çareniz,NaN,NaN,NaN,NaN,çareniz basit
8159,çarp,NaN,NaN,NaN,NaN,çarp bana
8160,şampiyona,NaN,NaN,NaN,NaN,şampiyon şampiyona


In [33]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,bana,2659182,"bana biraz, bana şans, bana asla, telefonu ban...","bana biraz şeker, bana telefon numaranı, bana ...","çekmiş bana elektronik postayla, orkestra şefi...",NaN,"bana biraz, bana çarptı, bana çekmiş, bana ası..."
1,biraz,1269641,"bana biraz, biraz fazla, biraz kahve, biraz mü...","bana biraz şeker, bana biraz fazla, bana biraz...","bazen motora biraz gaz, psikoloji branşını bir...",NaN,"biraz komik, biraz fazla, biraz kahve, bana bi..."
2,fazla,692043,"biraz fazla, bana fazla, dolardan fazla, milyo...","milyon dolardan fazla, bana biraz fazla, milyo...","fazla kaliteli normal banka, fazla profesyonel...",NaN,"biraz fazla, fazla riskli, fazla stresli, fazl..."
3,asla,463224,"bana asla, asla bana, asla kontrol, asla dans,...","bana asla asla, asla kredi kartını, asla manev...","fazla profesyonel maçında asla, profesyonel ma...",NaN,"asla doktor, asla leydim, asla lordum, asla ma..."
4,kontrol,272328,"numarayı kontrol, trafik kontrol, asla kontrol...","kredi kartını kontrol, modülü reaksiyon kontro...","servis modülü reaksiyon kontrol, şifreli bantl...","servis modülü reaksiyon kontrol sistemi, modül...","rutin kontrol, manuel kontrol, mikrofon kontro..."
...,...,...,...,...,...,...,...
8337,marti,5,"parti marti, parti marti",NaN,NaN,NaN,NaN
8338,faktörleri,5,"stres faktörleri, stres faktörleri",NaN,NaN,NaN,NaN
8339,ralli,5,"ralli kursuna, ralli kursuna",NaN,NaN,NaN,NaN
8340,penguenleri,5,"mutant penguenleri, çaresiz penguenleri, pengu...",NaN,NaN,NaN,NaN


In [34]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency.xlsx", index=False)

#### Copy Move And Delete

In [35]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*.xlsx")
output_file

['Turkish_English_Shared_Twogram.xlsx',
 'Turkish_English_Shared_Threegram.xlsx',
 'Turkish_English_Shared_Fourgram.xlsx',
 'Turkish_English_Shared_Fivegram.xlsx',
 'Turkish_English_Shared_Sentence.xlsx',
 'Turkish_English_Shared_Result_With_Frequency.xlsx',
 'Turkish_English_Shared_Join_Result_Without_Frequency.xlsx',
 'Turkish_English_Shared_Word_Prefix_Suffix_Custom.xlsx']

In [36]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [37]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass